## AB Test: Check how persona has effect on rec sys

In [24]:
# builtin
import importlib
# internal
import read_data
import train_model
import params
from tqdm.contrib.concurrent import process_map
# external
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

### Check the item name recommend to each user

In [25]:
res_dir = "experiment_result/MBA/"
lgcn_dir = res_dir + "LGCN/"
tri_lgcn_dir = res_dir + "LGCN_tri/"
tri_lgcn_approach_dir = res_dir + "LGCN_tri_approach/"
AB_test_dir = res_dir + "AB_test/"

In [26]:
# load MBA_LGCN_result.json and MBA_LGCN_tri_result.json
with open(AB_test_dir + 'MBA_LGCN_result.json', 'r') as f:
    lgcn_res = json.load(f)
with open(AB_test_dir + 'MBA_LGCN_tri_result.json', 'r') as f:
    lgcn_tri_res = json.load(f)

In [27]:
len(lgcn_res), len(lgcn_tri_res)

(4207, 4207)

In [28]:
# 把json按user_id的int排序，并检查是否一致
lgcn_res_sorted = {int(k): v for k,v in lgcn_res.items()}
lgcn_tri_res_sorted = {int(k): v for k,v in lgcn_tri_res.items()}
assert sorted(lgcn_res.keys()) == sorted(lgcn_tri_res.keys())

In [29]:
list_empty = []
for i in range(1, 4297):
    if i not in lgcn_res_sorted:
        list_empty.append(i)
# 导入tri_graph_uid2tidx_valid.json，检查list_empty里的user_id在这个json里对应的value是否为空
import json
with open('dataset/MBA/tri_graph_uidx2tidx_valid.json', 'r') as f:
    uid2tidx = json.load(f)
# 检查list_empty里的user_id在uid2tidx里对应的value是否都为空
for i in list_empty:
    assert uid2tidx[str(i)] == []


In [30]:
# load dataset
mba_df = pd.read_csv('dataset/MBA/market.csv', sep=';')
# mba_df = pd.read_csv('../ds/MBA/data.csv', sep=';')
print(mba_df.head())
# clean nan rows
if mba_df.isna().sum().sum() > 0:
    print('all nan eliminated')
    mba_df = mba_df.dropna()
# transfer types
mba_df['BillNo'] = mba_df['BillNo'].astype('int32')
mba_df['Itemname'] = mba_df['Itemname'].astype('string')
mba_df['Quantity'] = mba_df['Quantity'].astype('int32')
mba_df['Date'] = mba_df['Date'].astype('string')
mba_df['Price'] = mba_df['Price'].astype('string')
mba_df['CustomerID'] = mba_df['CustomerID'].astype('int32')

# Identifications
# for user nodes
user_ids = mba_df['CustomerID'].unique()
user_num = len(user_ids)
print(f'totally {user_num} unique users')
user_ids.sort()
user_ids_kv = {}
for ui in range(user_num):
    user_ids_kv[user_ids[ui]] = ui

# for item nodes
item_names = mba_df['Itemname'].unique()
item_num = len(item_names)
print(f'totally {item_num} unique items')
# item_names.sort()
items_kv = {}
for ii in range(item_num):
    items_kv[item_names[ii]] = ii

# for transaction nodes
trans_ids = mba_df['BillNo'].unique()
trans_num = len(trans_ids)
print(f'totally {trans_num} unique transactions')
trans_ids.sort()
trans_kv = {}
for ti in range(trans_num):
    trans_kv[trans_ids[ti]] = ti

# # construct the bi-partite graph
G_user = {} # {uidx: [tidx,]}
G_item = {} # {tidx: [uidx,]}

for index,row in mba_df.iterrows():
    user_index = user_ids_kv[row['CustomerID']]
    item_index = items_kv[row['Itemname']]
    
    # update user side
    if G_user.get(user_index) is None:
        G_user[user_index] = {item_index}
    else:
        G_user[user_index].update([item_index])
    
    # update item side
    if G_item.get(item_index) is None:
        G_item[item_index] = {user_index}
    else:
        G_item[item_index].update([user_index])

assert len(G_item.keys()) == item_num and len(G_user.keys()) == user_num

/var/folders/lh/1l1yxhh50vdcdltw404st9z80000gn/T/ipykernel_63854/2486020301.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  mba_df = pd.read_csv('dataset/MBA/market.csv', sep=';')


   BillNo                             Itemname  Quantity              Date  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  01.12.2010 08:26   
1  536365                  WHITE METAL LANTERN         6  01.12.2010 08:26   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  01.12.2010 08:26   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  01.12.2010 08:26   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  01.12.2010 08:26   

  Price  CustomerID         Country  
0  2,55     17850.0  United Kingdom  
1  3,39     17850.0  United Kingdom  
2  2,75     17850.0  United Kingdom  
3  3,39     17850.0  United Kingdom  
4  3,39     17850.0  United Kingdom  
all nan eliminated
totally 4297 unique users
totally 3846 unique items
totally 18163 unique transactions


In [31]:
item_names[1]

'WHITE METAL LANTERN'

In [32]:
# 把MBA_LGCN_result_sorted里，第二个和第三个[]里的item_id转换成item_name，也就是test_list和pred_list
lgcn_res_sorted_item_name = {}
for k, v in lgcn_res_sorted.items():
    v[1] = [item_names[i] for i in v[1]]
    v[2] = [item_names[i] for i in v[2]]
    lgcn_res_sorted_item_name[k] = v

# 把MBA_LGCN_tri_result_sorted里，第二个和第三个[]里的item_id转换成item_name，也就是test_list和pred_list
lgcn_tri_res_sorted_item_name = {}
for k, v in lgcn_tri_res_sorted.items():
    v[1] = [item_names[i] for i in v[1]]
    v[2] = [item_names[i] for i in v[2]]
    lgcn_tri_res_sorted_item_name[k] = v

# # 重新写进json
# with open(AB_test_dir + 'MBA_LGCN_result_sorted_item_name.json', 'w') as f:
#     json.dump(lgcn_res_sorted_item_name, f)
# with open(AB_test_dir + 'MBA_LGCN_tri_result_sorted_item_name.json', 'w') as f:
#     json.dump(lgcn_tri_res_sorted_item_name, f)

### Check the difference of LGCN recommend items and LGCN_tri recommend items

In [33]:
persona2idx_whole = {'Kitchen and Culinary Enthusiast': 0,
 'Home Decor Aficionado': 1,
 'Vintage and Retro Enthusiast': 2,
 'Seasonal and Festive Decorator': 3,
 'Crafting and DIY Hobbyist': 4,
 'Baking and Confectionery Fans': 5,
 'Child and Family-centric Shopper': 6,
 'Collector of Unique and Novelty Items': 7,
 'Comfort and Coziness Seeker': 8,
 'Gardening Lover': 9,
 'Tea and Coffee Lover': 10,
 'Educational and Learning Driven': 11,
 'Stationery and Organization Enthusiast': 12,
 'Eco-friendly and Sustainable Goods Supporter': 13,
 'Party and Event Planners': 14,
 'Outdoor and Adventure Prep': 15,
 'Fashion Forward and Accessory Savvy': 16,
 'Art and Design Admirers': 17,
 'Animal and Pet Advocates': 18,
 'Health and Wellness Focus': 19}

In [34]:
# 读入tri_graph_uidx2pidx.json，以便知道每个user被assign了哪些persona
with open('dataset/MBA/tri_graph_uidx2pidx.json', 'r') as f:
    uid2pidx = json.load(f)
# 按user_id排序
uid2pidx_sorted = {int(k): v for k,v in uid2pidx.items()}

# 再读入tri_graph_tidx2pidx.json，以便知道每个item有哪些persona
with open('dataset/MBA/tri_graph_tidx2pidx.json', 'r') as f:
    tidx2pidx = json.load(f)
# 按item_id排序
tidx2pidx_sorted = {int(k): v for k,v in tidx2pidx.items()}

In [233]:
len(uid2pidx), len(tidx2pidx), len(uid2pidx_sorted), len(tidx2pidx_sorted)

(4286, 3846, 4286, 3846)

In [36]:
# 对于lgcn_res_sorted_item_name和lgcn_tri_res_sorted_item_name，看看他们的pred_list里有哪些不一样的item_name
# 对于每个user，记录item_in_lgcn_not_in_lgcn_tri和item_in_lgcn_tri_not_in_lgcn

item_in_lgcn_not_in_lgcn_tri = {}
item_in_lgcn_tri_not_in_lgcn = {}
for k in lgcn_res_sorted_item_name.keys():
    v1 = lgcn_res_sorted_item_name[k]
    v2 = lgcn_tri_res_sorted_item_name[k]
    item_in_lgcn_not_in_lgcn_tri[k] = [i for i in v1[2] if i not in v2[2]]
    item_in_lgcn_tri_not_in_lgcn[k] = [i for i in v2[2] if i not in v1[2]]

# # 保存到json
# with open(AB_test_dir + 'item_in_lgcn_not_in_lgcn_tri.json', 'w') as f:
#     json.dump(item_in_lgcn_not_in_lgcn_tri, f)
# with open(AB_test_dir + 'item_in_lgcn_tri_not_in_lgcn.json', 'w') as f:
#     json.dump(item_in_lgcn_tri_not_in_lgcn, f)

In [37]:
# 找出在lgcn和lgcn_tri两个方法中被推荐item区别最大的user，也就是说，对于这个user，它在item_in_lgcn_not_in_lgcn_tri和item_in_lgcn_tri_not_in_lgcn里的item数量总数最多
max_diff_user = -1
max_diff_num = 0
for k in item_in_lgcn_not_in_lgcn_tri.keys():
    diff_num = len(item_in_lgcn_not_in_lgcn_tri[k]) + len(item_in_lgcn_tri_not_in_lgcn[k])
    if diff_num > max_diff_num:
        max_diff_num = diff_num
        max_diff_user = k

max_diff_user, max_diff_num


(4048, 198)

In [38]:
# 看看这个user的persona是什么
uid2pidx_sorted[max_diff_user]

[7]

In [39]:
# 把这个用户的item_in_lgcn_not_in_lgcn_tri和item_in_lgcn_tri_not_in_lgcn里的item_name打印出来
# item_in_lgcn_not_in_lgcn_tri[max_diff_user], item_in_lgcn_tri_not_in_lgcn[max_diff_user]
# item_in_lgcn_not_in_lgcn_tri[5]
item_in_lgcn_tri_not_in_lgcn[5]

['BLACK KITCHEN SCALES',
 'RED HANGING HEART T-LIGHT HOLDER',
 'PAPER LANTERN 9 POINT HOLLY STAR S',
 'CAKE PLATE LOVEBIRD WHITE',
 'RED KITCHEN SCALES',
 'ROUND CAKE TIN VINTAGE RED',
 'PIECE OF CAMO STATIONERY SET',
 'GRAND CHOCOLATECANDLE',
 'CAKE PLATE LOVEBIRD PINK',
 'BREAD BIN DINER STYLE IVORY',
 'PINK CREAM FELT CRAFT TRINKET BOX',
 'BISCUIT TIN VINTAGE RED',
 'WHITE HANGING HEART T-LIGHT HOLDER',
 'BOX OF 6 CHRISTMAS CAKE DECORATIONS',
 'IVORY KITCHEN SCALES',
 'SET 3 SONG BIRD PAPER EGGS ASSORTED',
 'ROUND CAKE TIN VINTAGE GREEN',
 'CUPCAKE LACE PAPER SET 6',
 'SWEETHEART CAKESTAND 3 TIER',
 'SET/4 SPRING FLOWER DECORATION',
 'ASSORTED TUTTI FRUTTI LARGE PURSE',
 'GREEN SQUARE COMPACT MIRROR',
 '72 SWEETHEART FAIRY CAKE CASES',
 'ASSORTED TUTTI FRUTTI PEN',
 'NINE DRAWER OFFICE TIDY',
 'BREAD BIN DINER STYLE MINT',
 'LANTERN CREAM GAZEBO',
 'PINK BLUE FELT CRAFT TRINKET BOX',
 'ROSES REGENCY TEACUP AND SAUCER',
 'JET BLACK LAS VEGAS NECKLACE 45CM',
 'PINK JEWELLED PHOTO FRAM

In [40]:
# 看item_in_lgcn_tri_not_in_lgcn[5][0]对应的persona是什么
# position_list = [3, 5, 8, 9, 10, 11, 13, 16, 22, 25, 26, 33, 34, 41]
count_lgcn = 0
count_lgcn_tri = 0
for i in range (0, 99):
    print(f'{i}: {tidx2pidx_sorted[items_kv[item_in_lgcn_not_in_lgcn_tri[4048][i]]]}')
    print(f'{i}: {tidx2pidx_sorted[items_kv[item_in_lgcn_tri_not_in_lgcn[4048][i]]]}')
    if 7 in tidx2pidx_sorted[items_kv[item_in_lgcn_not_in_lgcn_tri[4048][i]]]:
        count_lgcn += 1
    if 7 in tidx2pidx_sorted[items_kv[item_in_lgcn_tri_not_in_lgcn[4048][i]]]:
        count_lgcn_tri += 1
# tidx2pidx_sorted[items_kv[item_in_lgcn_tri_not_in_lgcn[5][3]]]
print(count_lgcn, count_lgcn_tri)



0: [2, 7]
0: [8, 4, 16]
1: [3, 7]
1: [9, 1, 18]
2: [8, 2, 7]
2: [1, 4]
3: [12, 6, 11]
3: [18, 1, 7]
4: [2, 7, 1]
4: [10, 18, 1, 7]
5: [12, 7, 4]
5: [18, 7, 2]
6: [4, 3, 7, 16, 17]
6: [9, 1, 18]
7: [1, 2, 3]
7: [7, 6, 18]
8: [1, 8, 7]
8: [4, 11, 17]
9: [12, 4, 6, 14, 7]
9: [7, 1, 18]
10: [2, 7, 1]
10: [4, 18, 7]
11: [0, 2]
11: [18, 12, 7]
12: [0, 1, 7]
12: [18, 4, 7]
13: [7, 6]
13: [18, 1]
14: [8, 2]
14: [7, 18, 1]
15: [8, 7]
15: [1, 3, 18, 7]
16: [16, 7, 2]
16: [18, 1, 7]
17: [14, 8, 6]
17: [1, 18, 7]
18: [7, 16]
18: [6, 18, 7]
19: [8, 7, 2]
19: [6, 8, 18]
20: []
20: [18, 1, 7]
21: [8, 7]
21: [16, 7]
22: [2, 7, 8]
22: [7, 18]
23: [1, 2, 4]
23: [1, 18, 7]
24: [2, 16, 7]
24: [1, 9, 2, 4]
25: [1, 2, 4]
25: [6, 7, 8, 3, 18]
26: [3, 1, 2]
26: [9, 1, 7]
27: [8, 18, 2, 6]
27: [1, 2, 0]
28: [1, 3, 9]
28: [4, 6, 18, 7]
29: [4, 0, 1, 2]
29: [1, 8]
30: [8]
30: [1, 9, 7]
31: [4, 3, 14]
31: [18, 0, 7, 8]
32: [4, 12, 11, 17]
32: [3, 1, 4]
33: [1, 3, 8]
33: [1, 18, 7]
34: [1, 17]
34: [18, 7]
35: [0, 

### Narrow down user range: 5 purchased items, top 20, overlap percentage inceases most: 2/20->5/20;

In [41]:
overlap_count_lgcn = {}
overlap_percentage_lgcn = {}
overlap_count_lgcn_tri = {}
overlap_percentage_lgcn_tri = {}
incresed_percentage = {}

for k in lgcn_res_sorted_item_name.keys():
# for k in range (1, 2):
    v1 = lgcn_res_sorted_item_name[k]
    v2 = lgcn_tri_res_sorted_item_name[k]
    # print(set(v1[2][:20]))
    # print(set(v1[1][0:]))
    overlap_count_lgcn[k] = len(set(v1[2][:20]) & set(v1[1][0:]))
    overlap_percentage_lgcn[k] = overlap_count_lgcn[k] / len(set(v1[1][0:]))
    # print(overlap_percentage_lgcn[k])
    overlap_count_lgcn_tri[k] = len(set(v2[2][:20]) & set(v2[1][0:]))
    overlap_percentage_lgcn_tri[k] = overlap_count_lgcn_tri[k] / len(set(v2[1][0:]))
    # print(overlap_percentage_lgcn_tri[k])
    incresed_percentage[k] = overlap_percentage_lgcn_tri[k] - overlap_percentage_lgcn[k]

# 找出increase_percentage最大的100个user
sorted_incresed_percentage = sorted(incresed_percentage.items(), key=lambda x: x[1], reverse=True)
# sorted_incresed_percentage[:100]
# 把这些user写进一个list
top_100_incresed_percentage = [k for k, v in sorted_incresed_percentage[:100]]
print(top_100_incresed_percentage)

[191, 383, 679, 833, 1008, 1631, 1964, 1975, 2070, 2190, 2259, 2354, 3038, 3063, 3281, 3285, 3658, 3759, 3938, 4047, 4254, 37, 3726, 484, 1247, 2377, 2708, 3065, 3258, 68, 71, 103, 115, 230, 435, 461, 479, 572, 690, 731, 811, 857, 972, 986, 1037, 1054, 1212, 1222, 1298, 1370, 1419, 1501, 1546, 1638, 1686, 1720, 2138, 2152, 2201, 2219, 2224, 2419, 2456, 2472, 2498, 2536, 2597, 2643, 2774, 2813, 3093, 3251, 3311, 3405, 3428, 3701, 3743, 3750, 3885, 3949, 4059, 4125, 4164, 505, 581, 751, 1630, 2127, 2801, 3334, 4106, 960, 1141, 2074, 236, 3719, 777, 906, 1659, 2216]


### For those newly corrected chosen items, check if they are related with assigned personas;

In [42]:
new_correct_items = {}
removed_wrong_items = {}

for k in top_100_incresed_percentage:
    v1 = lgcn_res_sorted_item_name[k]
    v2 = lgcn_tri_res_sorted_item_name[k]
    new_correct_items[k] = [i for i in v2[2][:20] if i not in v1[2][:20] and i in v2[1]]
    removed_wrong_items[k] = [i for i in v1[2][:20] if i not in v2[2][:20] and i not in v1[1]]

In [43]:
# new_correct_items
# 检查这些new_correct_items里的item被assign的persona，以及有多少item被assign的persona也是这个user的persona
persona_related_item_count = {}
# new_correct_items_persona_count
for k in top_100_incresed_percentage:
    # print(f'{k}: {uid2pidx_sorted[k]}')
    for i in new_correct_items[k]:
        # print(f'{i}: {tidx2pidx_sorted[items_kv[i]]}')
        if len(set(tidx2pidx_sorted[items_kv[i]]) & set(uid2pidx_sorted[k])) > 0:
            persona_related_item_count[k] = persona_related_item_count.get(k, 0) + 1
            persona_related_item_count[k] = persona_related_item_count[k] / len(new_correct_items[k]) # ratio

# 算一下平均的ratio
average_ratio_related_with_persona = sum(persona_related_item_count.values()) / len(persona_related_item_count)
print(average_ratio_related_with_persona)
# len(persona_related_item_count)
# persona_related_item_count

0.8792260599415205


In [79]:
# 把这些new correct对于每个user的item写进json
with open(AB_test_dir + 'new_correct_items.json', 'w') as f:
    json.dump(new_correct_items, f)

In [ ]:
# removed_wrong_items

### Check about top 20 occurred items

In [68]:
# 从G_item里找出最多的20个item
item_count = {}
for k in G_item.keys():
    item_count[k] = len(G_item[k])
sorted_item_count = sorted(item_count.items(), key=lambda x: x[1], reverse=True)
# sorted_item_count[:20]
# 把最多的20个item写进一个list
overall_top_20_item = [k for k, v in sorted_item_count[:20]]
# 然后把id转换成name
overall_top_20_item_name = [item_names[i] for i in overall_top_20_item]
# print(overall_top_20_item_name)

['REGENCY CAKESTAND 3 TIER', 'WHITE HANGING HEART T-LIGHT HOLDER', 'PARTY BUNTING', 'ASSORTED COLOUR BIRD ORNAMENT', 'SET OF 3 CAKE TINS PANTRY DESIGN', 'PACK OF 72 RETROSPOT CAKE CASES', 'JUMBO BAG RED RETROSPOT', "PAPER CHAIN KIT 50'S CHRISTMAS", 'NATURAL SLATE HEART CHALKBOARD', 'BAKING SET 9 PIECE RETROSPOT', 'HEART OF WICKER SMALL', 'SPOTTY BUNTING', 'JAM MAKING SET WITH JARS', 'JAM MAKING SET PRINTED', 'LUNCH BAG RED RETROSPOT', 'LUNCH BAG SPACEBOY DESIGN', 'RETROSPOT TEA SET CERAMIC 11 PC', 'HEART OF WICKER LARGE', 'REX CASH+CARRY JUMBO SHOPPER', 'LUNCH BAG CARS BLUE']


In [81]:
# 把这些removed wrong对于每个user的item写进json
with open(AB_test_dir + 'removed_wrong_popular_items.json', 'w') as f:
    # 如果这些remove_wrong item是最热门的20个item之一，则写进json，否则不写
    removed_wrong_items_top_20 = {}
    for k in removed_wrong_items.keys():
        removed_wrong_items_top_20[k] = [i for i in removed_wrong_items[k] if i in overall_top_20_item_name]
    json.dump(removed_wrong_items_top_20, f)

In [65]:
# 首先检查对于lgcn_res_sorted_item_name的top20预测，有多少正确的item在overall_top_20_item_name里，有多少不正确的item在overall_top_20_item里
correct_item_in_top_20_item = {}
wrong_item_in_top_20_item = {}
# for k in top_100_incresed_percentage:
for k in lgcn_res_sorted_item_name.keys():
    v1 = lgcn_res_sorted_item_name[k]
    correct_item_in_top_20_item[k] = [i for i in v1[2][:20] if i in overall_top_20_item_name and i in v1[1]]
    wrong_item_in_top_20_item[k] = [i for i in v1[2][:20] if i in overall_top_20_item_name and i not in v1[1]]

# correct_item_in_top_20_item
count_correct = 0
for k in correct_item_in_top_20_item.keys():
    count_correct += len(correct_item_in_top_20_item[k])
print(count_correct)

# wrong_item_in_top_20_item
count_wrong = 0
for k in wrong_item_in_top_20_item.keys():
    count_wrong += len(wrong_item_in_top_20_item[k])
print(count_wrong)

1192
7287


In [66]:
# 类似的对于lgcn_tri_res_sorted_item_name的top20预测，有多少正确的item在overall_top_20_item_name里，有多少不正确的item在overall_top_20_item里
correct_item_in_top_20_item_tri = {}
wrong_item_in_top_20_item_tri = {}
# for k in top_100_incresed_percentage:
for k in lgcn_tri_res_sorted_item_name.keys():
    v2 = lgcn_tri_res_sorted_item_name[k]
    correct_item_in_top_20_item_tri[k] = [i for i in v2[2][:20] if i in overall_top_20_item_name and i in v2[1]]
    wrong_item_in_top_20_item_tri[k] = [i for i in v2[2][:20] if i in overall_top_20_item_name and i not in v2[1]]
13
# correct_item_in_top_20_item_tri
count_correct_tri = 0
for k in correct_item_in_top_20_item_tri.keys():
    count_correct_tri += len(correct_item_in_top_20_item_tri[k])
print(count_correct_tri)
# wrong_item_in_top_20_item_tri
count_wrong_tri = 0
for k in wrong_item_in_top_20_item_tri.keys():
    count_wrong_tri += len(wrong_item_in_top_20_item_tri[k])
print(count_wrong_tri)

1111
5973


### Specific Case Study:

In [91]:
# Part 1: New Correct Items Due to Right Persona

# Case 0:
# user 572 persona
print(uid2pidx_sorted[572]) # [1, 3, 9, 14]
# new correct items
print(tidx2pidx_sorted[items_kv["TEA TIME PARTY BUNTING"]]) # [3, 14, 10] -> 3, 14
# where 3 is "Seasonal and Festive Decorator" and 14 is "Party and Event Planners"
print("====================================")

# Case 1:
# user 960 persona
print(uid2pidx_sorted[960]) # [2, 3, 4, 5, 6]
# new correct items
print(tidx2pidx_sorted[items_kv["PENNY FARTHING BIRTHDAY CARD"]]) # [2, 7, 12] -> 2
print(tidx2pidx_sorted[items_kv["CARD WEDDING DAY"]]) # [14, 7, 4] -> 4
# where 2 is "Vintage and Retro Enthusiast" and 4 is "Crafting and DIY Hobbyist"
print("====================================")


[1, 3, 9, 14]
[3, 14, 10]
[3, 2, 4, 5, 6]
[2, 7, 12]
[14, 7, 4]


In [92]:
# Part 2: Wrong Removed Popular Items

# Case 0:
# user 572 persona
print(uid2pidx_sorted[572]) # [1, 3, 9, 14]
# removed wrong items
print(tidx2pidx_sorted[items_kv["ASSORTED COLOUR BIRD ORNAMENT"]]) # [1, 18, 7, 9] -> 7, 18
print(tidx2pidx_sorted[items_kv["HEART OF WICKER LARGE"]]) # [1, 2, 4] -> 2, 4
# where 2 is "Vintage and Retro Enthusiast", 4 is "Crafting and DIY Hobbyist", 7 is "Child and Family-centric Shopper" and 18 is "Animal and Pet Advocates"
print("====================================")

# Case 1:
# user 479 persona
print(uid2pidx_sorted[479]) # [1, 5, 7]
# removed wrong items
print(tidx2pidx_sorted[items_kv["NATURAL SLATE HEART CHALKBOARD"]]) # [1, 4, 12] -> 5, 7
# where 5 is "Baking and Confectionery Fans" and 7 is "Child and Family-centric Shopper"
print("====================================")





[1, 3, 9, 14]
[1, 18, 7, 9]
[1, 2, 4]
[1, 5, 7]
[1, 4, 12]


In [108]:
# Part 3: Narrow down a range for user: only select those with 5-10 items in the test set
# 选择test set里item数量在5-10之间的user，以及他们在lgcn_res_sorted_item_name和lgcn_tri_res_sorted_item_name里的item，保存为两个json
user_item_count = {}
for k in lgcn_res_sorted_item_name.keys():
    user_item_count[k] = len(lgcn_res_sorted_item_name[k][1])
# user_item_count
selected_user = [k for k, v in user_item_count.items() if v >= 10 and v <= 10]
len(selected_user)

selected_lgcn_res_sorted_item_name = {k: lgcn_res_sorted_item_name[k] for k in selected_user}
selected_lgcn_tri_res_sorted_item_name = {k: lgcn_tri_res_sorted_item_name[k] for k in selected_user}

# 保存到json
with open(AB_test_dir + 'selected_lgcn_res_sorted_item_name.json', 'w') as f:
    json.dump(selected_lgcn_res_sorted_item_name, f)
with open(AB_test_dir + 'selected_lgcn_tri_res_sorted_item_name.json', 'w') as f:
    json.dump(selected_lgcn_tri_res_sorted_item_name, f)


In [109]:
# 看看user一致不一致
assert sorted(selected_lgcn_res_sorted_item_name.keys()) == sorted(selected_lgcn_tri_res_sorted_item_name.keys())

In [110]:
# Part 4: 在这些test_list长度为10的case中，找到一个典型的user，对于这个user，看他的pred_list里的top 10，对比lgcn和lgcn_tri，看看有没有new correct items和removed wrong popular items

# 首先看看这些user在lgcn和lgcn_tri里的top 10有多少overlap
overlap_count_lgcn_10 = {}
overlap_percentage_lgcn_10 = {}
overlap_count_lgcn_tri_10 = {}
overlap_percentage_lgcn_tri_10 = {}
incresed_percentage_10 = {}

for k in selected_lgcn_res_sorted_item_name.keys():
    v1 = selected_lgcn_res_sorted_item_name[k]
    v2 = selected_lgcn_tri_res_sorted_item_name[k]
    # print(set(v1[2][:20]))
    # print(set(v1[1][0:]))
    overlap_count_lgcn_10[k] = len(set(v1[2][:10]) & set(v1[1][0:]))
    overlap_percentage_lgcn_10[k] = overlap_count_lgcn_10[k] / len(set(v1[1][0:]))
    # print(overlap_percentage_lgcn[k])
    overlap_count_lgcn_tri_10[k] = len(set(v2[2][:10]) & set(v2[1][0:]))
    overlap_percentage_lgcn_tri_10[k] = overlap_count_lgcn_tri_10[k] / len(set(v2[1][0:]))
    # print(overlap_percentage_lgcn_tri[k])
    incresed_percentage_10[k] = overlap_percentage_lgcn_tri_10[k] - overlap_percentage_lgcn_10[k]

# 找出increase_percentage最大的100个user
sorted_incresed_percentage_10 = sorted(incresed_percentage_10.items(), key=lambda x: x[1], reverse=True)
print(sorted_incresed_percentage_10[:100])

# 把这些user写进一个list
top_100_incresed_percentage_10 = [k for k, v in sorted_incresed_percentage_10[:100]]
# print(top_100_incresed_percentage_10)

[(2004, 0.3), (231, 0.2), (701, 0.2), (1729, 0.2), (2910, 0.2), (84, 0.19999999999999998), (954, 0.19999999999999998), (2293, 0.10000000000000003), (2455, 0.10000000000000003), (2577, 0.10000000000000003), (59, 0.1), (99, 0.1), (284, 0.1), (356, 0.1), (524, 0.1), (622, 0.1), (689, 0.1), (853, 0.1), (1026, 0.1), (1251, 0.1), (1626, 0.1), (1941, 0.1), (2085, 0.1), (2098, 0.1), (2240, 0.1), (2263, 0.1), (2394, 0.1), (2507, 0.1), (3255, 0.1), (3325, 0.1), (3379, 0.1), (3444, 0.1), (3770, 0.1), (3829, 0.1), (4003, 0.1), (680, 0.09999999999999998), (30, 0.0), (35, 0.0), (97, 0.0), (204, 0.0), (266, 0.0), (279, 0.0), (309, 0.0), (322, 0.0), (371, 0.0), (475, 0.0), (564, 0.0), (699, 0.0), (744, 0.0), (761, 0.0), (789, 0.0), (885, 0.0), (947, 0.0), (950, 0.0), (973, 0.0), (975, 0.0), (983, 0.0), (984, 0.0), (1077, 0.0), (1231, 0.0), (1305, 0.0), (1348, 0.0), (1465, 0.0), (1629, 0.0), (1811, 0.0), (1814, 0.0), (1824, 0.0), (1888, 0.0), (1948, 0.0), (1962, 0.0), (2006, 0.0), (2084, 0.0), (2150, 0

In [148]:
# Part 5: 具体看看2004这个user和其推荐情况

# 打印2004这个user的test_list
print("For user 2004, items that he really purchased are:" + str(selected_lgcn_res_sorted_item_name[2004][1]))
# 再打印2004这个user的pred_list（使用LGCN的top 10）
print("For user 2004, items that LGCN recommended are:" + str(selected_lgcn_res_sorted_item_name[2004][2][:10]))
# 再打印2004这个user的pred_list（使用LGCN_tri的top 10）
print("For user 2004, items that LGCN_tri recommended are:" + str(selected_lgcn_tri_res_sorted_item_name[2004][2][:10]))


# user 2004 persona
print("And user 2004 has persona: " + str(uid2pidx_sorted[2004])) # [5, 3, 9, 14, 8, 7]
# where 5 is "Baking and Confectionery Fans", 3 is "Seasonal and Festive Decorator";
# 7 is "Child and Family-centric Shopper", 8 is "Gardening Lover", 
# 9 is "Comfort and Coziness Seeker" and 14 is "Party and Event Planners"

# 首先找出对于2004有哪些new correct items
new_correct_items_2004 = [i for i in selected_lgcn_tri_res_sorted_item_name[2004][2][:10] if i not in selected_lgcn_res_sorted_item_name[2004][2][:10] and i in selected_lgcn_tri_res_sorted_item_name[2004][1][:10]]
print("For user 2004, new correct items are:" + str(new_correct_items_2004))
# 再找出对于2004有哪些removed wrong popular items
removed_wrong_items_2004 = [i for i in selected_lgcn_res_sorted_item_name[2004][2][:10] if i not in selected_lgcn_tri_res_sorted_item_name[2004][2][:10] and i not in selected_lgcn_res_sorted_item_name[2004][1][:10] and i in overall_top_20_item_name]
print("For user 2004, removed wrong popular items are:" + str(removed_wrong_items_2004))

# new correct items
print("NEW CORRECT ITEMS:")
print("Item: LAUREL HEART ANTIQUE SILVER: " + str(tidx2pidx_sorted[items_kv["LAUREL HEART ANTIQUE SILVER"]])) # [2, 16, 7] -> 7
print("And this item is related with user 2004's persona: 'Child and Family-centric Shopper'") 
print("Item: CHERUB HEART DECORATION GOLD: " + str(tidx2pidx_sorted[items_kv["CHERUB HEART DECORATION GOLD"]])) # [1, 3, 2, 7] -> 3, 7
print("And this item is related with user 2004's persona: 'Seasonal and Festive Decorator' and 'Child and Family-centric Shopper'")
print("Item: RED HANGING HEART T-LIGHT HOLDER: " + str(tidx2pidx_sorted[items_kv["RED HANGING HEART T-LIGHT HOLDER"]])) # [1, 2, 8, 3] -> 3, 8
print("And this item is related with user 2004's persona: 'Seasonal and Festive Decorator' and 'Gardening Lover'")
# where 3 is "Seasonal and Festive Decorator" and 7 is "Child and Family-centric Shopper" and 8 is "Gardening Lover"

# removed wrong items
print("REMOVED WRONG ITEMS:")
print("Item: JUMBO BAG RED RETROSPOT: " + str(tidx2pidx_sorted[items_kv["JUMBO BAG RED RETROSPOT"]])) # [2, 7, 16] -> 2, 16
print("And this item is related with user 2004's persona: 'Vintage and Retro Enthusiast' and 'Art and Design Admirer'")
# where 2 is "Vintage and Retro Enthusiast" and 16 is "Art and Design Admirers"
print("====================================")


For user 2004, items that he really purchased are:['CHARLOTTE BAG PINK POLKADOT', 'PAPER BUNTING VINTAGE PAISLEY', 'CHERUB HEART DECORATION GOLD', 'LAUREL HEART ANTIQUE SILVER', 'RED HANGING HEART T-LIGHT HOLDER', 'ASSORTED COLOUR T-LIGHT HOLDER', 'VINTAGE SNAP CARDS', 'PAPER CHAIN KIT VINTAGE CHRISTMAS', 'DAISY GARDEN MARKER', 'ZINC FOLKART SLEIGH BELLS']
For user 2004, items that LGCN recommended are:['WOODEN HEART CHRISTMAS SCANDINAVIAN', 'LUNCH BAG  BLACK SKULL.', 'CHOCOLATE HOT WATER BOTTLE', 'JUMBO BAG RED RETROSPOT', 'HOMEMADE JAM SCENTED CANDLES', 'WOODEN STAR CHRISTMAS SCANDINAVIAN', 'WOODEN TREE CHRISTMAS SCANDINAVIAN', 'Next Day Carriage', 'PACK OF 72 SKULL CAKE CASES', 'STRAWBERRY CERAMIC TRINKET BOX']
For user 2004, items that LGCN_tri recommended are:['LAUREL HEART ANTIQUE SILVER', 'CHERUB HEART DECORATION GOLD', 'CHOCOLATE HOT WATER BOTTLE', 'LAUREL STAR ANTIQUE SILVER', 'SWISS ROLL TOWEL PINK  SPOTS', 'LAUREL STAR ANTIQUE GOLD', 'WHITE ROCKING HORSE HAND PAINTED', 'RED 

In [149]:
# 类似的看看701这个user和其推荐情况

# 打印701这个user的test_list
print("For user 701, items that he really purchased are:" + str(selected_lgcn_res_sorted_item_name[701][1]))
# 再打印701这个user的pred_list（使用LGCN的top 10）
print("For user 701, items that LGCN recommended are:" + str(selected_lgcn_res_sorted_item_name[701][2][:10]))
# 再打印701这个user的pred_list（使用LGCN_tri的top 10）
print("For user 701, items that LGCN_tri recommended are:" + str(selected_lgcn_tri_res_sorted_item_name[701][2][:10]))

# user 701 persona
print("And user 701 has persona: " + str(uid2pidx_sorted[701])) # [3, 5, 4, 2, 1, 9, 7]

# 首先找出对于701有哪些new correct items
new_correct_items_701 = [i for i in selected_lgcn_tri_res_sorted_item_name[701][2][:10] if i not in selected_lgcn_res_sorted_item_name[701][2][:10] and i in selected_lgcn_tri_res_sorted_item_name[701][1][:10]]
print("For user 701, new correct items are:" + str(new_correct_items_701))
# 再找出对于701有哪些removed wrong popular items
removed_wrong_items_701 = [i for i in selected_lgcn_res_sorted_item_name[701][2][:10] if i not in selected_lgcn_tri_res_sorted_item_name[701][2][:10] and i not in selected_lgcn_res_sorted_item_name[701][1][:10] and i in overall_top_20_item_name]
print("For user 701, removed wrong popular items are:" + str(removed_wrong_items_701))

# new correct items
print("NEW CORRECT ITEMS:")
print("Item: PINK FLORAL FELTCRAFT SHOULDER BAG: " + str(tidx2pidx_sorted[items_kv["PINK FLORAL FELTCRAFT SHOULDER BAG"]])) # [16, 4, 7] -> 4, 7
print("And this item is related with user 701's persona: 'Crafting and DIY Hobbyist' and 'Collector of Unique and Novelty Items'")
print("Item: FELTCRAFT PRINCESS OLIVIA DOLL: " + str(tidx2pidx_sorted[items_kv["FELTCRAFT PRINCESS OLIVIA DOLL"]])) # [4, 7, 6] -> 4, 7
print("And this item is related with user 701's persona: 'Crafting and DIY Hobbyist' and 'Collector of Unique and Novelty Items'")

# removed wrong items
print("REMOVED WRONG ITEMS:")
print("Item: REGENCY CAKESTAND 3 TIER: " + str(tidx2pidx_sorted[items_kv["REGENCY CAKESTAND 3 TIER"]])) # [5, 1, 2, 0, 10] -> 0, 10
print("And this item is related with user 701's persona: 'Kitchen and Culinary Enthusiast' and 'Tea and Coffee Lover'")
print("Item: JAM MAKING SET WITH JARS: " + str(tidx2pidx_sorted[items_kv["JAM MAKING SET WITH JARS"]])) # [4, 5, 0, 13] -> 0, 13
print("And this item is related with user 701's persona: 'Kitchen and Culinary Enthusiast' and 'Eco-friendly and Sustainable Goods Supporter'")
# print("Item: WHITE HANGING HEART T-LIGHT HOLDER: " + str(tidx2pidx_sorted[items_kv["WHITE HANGING HEART T-LIGHT HOLDER"]])) # [1, 2, 3] -> None
# where 0 is "Kitchen and Culinary Enthusiast" and 10 is "Tea and Coffee Lover" and 13 is "Eco-friendly and Sustainable Goods Supporter"

print("====================================")


For user 701, items that he really purchased are:['WOODEN BOX OF DOMINOES', 'CHRISTMAS CRAFT WHITE FAIRY', '3 STRIPEY MICE FELTCRAFT', 'PAPER BUNTING VINTAGE PAISLEY', 'PINK FLORAL FELTCRAFT SHOULDER BAG', 'FELTCRAFT PRINCESS OLIVIA DOLL', 'PACK OF 12 SUKI TISSUES', 'TRADITIONAL WOODEN SKIPPING ROPE', 'FELTCRAFT DOLL EMILY', 'WORLD WAR 2 GLIDERS ASSTD DESIGNS']
For user 701, items that LGCN recommended are:['PINK CREAM FELT CRAFT TRINKET BOX', '3 STRIPEY MICE FELTCRAFT', 'PINK BLUE FELT CRAFT TRINKET BOX', 'REGENCY CAKESTAND 3 TIER', 'FELTCRAFT DOLL MOLLY', 'JAM MAKING SET WITH JARS', 'PARTY BUNTING', 'FELTCRAFT DOLL EMILY', 'WHITE HANGING HEART T-LIGHT HOLDER', 'TRADITIONAL KNITTING NANCY']
For user 701, items that LGCN_tri recommended are:['PINK CREAM FELT CRAFT TRINKET BOX', 'PINK BLUE FELT CRAFT TRINKET BOX', 'PARTY BUNTING', 'FELTCRAFT DOLL MOLLY', '3 STRIPEY MICE FELTCRAFT', 'SPOTTY BUNTING', 'PINK FLORAL FELTCRAFT SHOULDER BAG', 'FELTCRAFT DOLL EMILY', 'FELTCRAFT PRINCESS OLIVIA

In [246]:
# 1. 这两个item的popularity在所有item中分别处于什么位置：首先把这两个item_name对应到index，然后在sorted_item_count里找到这两个item的位置
print("For item 'PINK FLORAL FELTCRAFT SHOULDER BAG':")
print(items_kv["PINK FLORAL FELTCRAFT SHOULDER BAG"])
# print(item_count[items_kv["PINK FLORAL FELTCRAFT SHOULDER BAG"]])
print("The popularity of this item is: " + str(item_count[items_kv["PINK FLORAL FELTCRAFT SHOULDER BAG"]]))
print(sorted_item_count[601])
print("====================================")

print("For item 'FELTCRAFT PRINCESS OLIVIA DOLL':")
print(items_kv["FELTCRAFT PRINCESS OLIVIA DOLL"])
# print(item_count[items_kv["FELTCRAFT PRINCESS OLIVIA DOLL"]])
print("The popularity of this item is: " + str(item_count[items_kv["FELTCRAFT PRINCESS OLIVIA DOLL"]]))
print(sorted_item_count[379])
print("====================================")


# print(overall_top_20_item_name)
# print("====================================")

# For original recommended popular 3 items:['REGENCY CAKESTAND 3 TIER', 'JAM MAKING SET WITH JARS', 'WHITE HANGING HEART T-LIGHT HOLDER']
print("For item 'REGENCY CAKESTAND 3 TIER':")
# print(items_kv["REGENCY CAKESTAND 3 TIER"])
# print(sorted_item_count[items_kv["REGENCY CAKESTAND 3 TIER"]])
print("The popularity of this item is: " + str(item_count[items_kv["REGENCY CAKESTAND 3 TIER"]]))
print("====================================")


print("For item 'JAM MAKING SET WITH JARS':")
# print(items_kv["JAM MAKING SET WITH JARS"])
# print(sorted_item_count[items_kv["JAM MAKING SET WITH JARS"]])
print("The popularity of this item is: " + str(item_count[items_kv["JAM MAKING SET WITH JARS"]]))
print("====================================")


print("For item 'WHITE HANGING HEART T-LIGHT HOLDER':")
# print(items_kv["WHITE HANGING HEART T-LIGHT HOLDER"])
# print(sorted_item_count[items_kv["WHITE HANGING HEART T-LIGHT HOLDER"]])
print("The popularity of this item is: " + str(item_count[items_kv["WHITE HANGING HEART T-LIGHT HOLDER"]]))
print("====================================")


# user 701
user_index = 701
# item 'PINK FLORAL FELTCRAFT SHOULDER BAG'
item_index = items_kv["PINK FLORAL FELTCRAFT SHOULDER BAG"]
# item 'FELTCRAFT PRINCESS OLIVIA DOLL'
item_index_2 = items_kv["FELTCRAFT PRINCESS OLIVIA DOLL"]

# user 701 <-> item 'PINK FLORAL FELTCRAFT SHOULDER BAG'
if G_user.get(user_index) is not None:
    if item_index in G_user[user_index]:
        print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are connected.")
    else:
        print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are not connected.")
print("====================================")

# user 701 <-> item 'FELTCRAFT PRINCESS OLIVIA DOLL'
if G_user.get(user_index) is not None:
    if item_index_2 in G_user[user_index]:
        print("User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected.")
    else:
        print("User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are not connected.")
print("====================================")

# 在train的graph里找到这两个item的连接的user，看有几跳
with open('dataset/MBA/tri_graph_uidx2tidx_train.json', 'r') as f:
    uidx2tidx_train = json.load(f)

print(item_index, item_index_2)




For item 'PINK FLORAL FELTCRAFT SHOULDER BAG':
1062
The popularity of this item is: 138
(1062, 138)
For item 'FELTCRAFT PRINCESS OLIVIA DOLL':
377
The popularity of this item is: 179
(377, 179)
For item 'REGENCY CAKESTAND 3 TIER':
The popularity of this item is: 869
For item 'JAM MAKING SET WITH JARS':
The popularity of this item is: 562
For item 'WHITE HANGING HEART T-LIGHT HOLDER':
The popularity of this item is: 844
User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are connected.
User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected.
1062 377


In [250]:
# 首先检查在uidx2tidx_train里，对于用户701，这两个item是否有连接
if uidx2tidx_train.get(str(user_index)) is not None:
    if item_index in uidx2tidx_train[str(user_index)]:
        print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are connected.")
    else:
        print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are not connected.")
print("====================================")

if uidx2tidx_train.get(str(user_index)) is not None:
    if item_index_2 in uidx2tidx_train[str(user_index)]:
        print("User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected.")
    else:
        print("User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are not connected.")
print("====================================")

# 但是可以检查uidx2tidx，是否在701的item里有别的item，这个item连接到另一个user，然后另一个user的item里有这两个item
items_for_701 = uidx2tidx_train[str(user_index)]
num_of_connection_user = 0

for item in items_for_701:
    for user in uidx2tidx_train.keys():
        if user != str(user_index):
            if item in uidx2tidx_train[user]:
                if item_index in uidx2tidx_train[user]: # and item_index_2 in uidx2tidx_train[user]:
                    # print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' and 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected through another user.")
                    # print("And this user is: " + str(user))
                    num_of_connection_user += 1

print("User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' and 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected through another user.")
print("And there are " + str(num_of_connection_user) + " users that connect them.")


User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' are not connected.
User 701 and item 'FELTCRAFT PRINCESS OLIVIA DOLL' are not connected.
User 701 and item 'PINK FLORAL FELTCRAFT SHOULDER BAG' and 'FELTCRAFT PRINCESS OLIVIA DOLL' are connected through another user.
And there are 758 users that connect them.


In [253]:
wrong_item_index_1 = items_kv["REGENCY CAKESTAND 3 TIER"]
wrong_item_index_2 = items_kv["JAM MAKING SET WITH JARS"]
wrong_item_index_3 = items_kv["WHITE HANGING HEART T-LIGHT HOLDER"]

# 再检查wrong item是不是和user 701有连接
if uidx2tidx_train.get(str(user_index)) is not None:
    if wrong_item_index_1 in uidx2tidx_train[str(user_index)]:
        print("User 701 and item 'REGENCY CAKESTAND 3 TIER' are connected.")
    else:
        print("User 701 and item 'REGENCY CAKESTAND 3 TIER' are not connected.")
print("====================================")

if uidx2tidx_train.get(str(user_index)) is not None:
    if wrong_item_index_2 in uidx2tidx_train[str(user_index)]:
        print("User 701 and item 'JAM MAKING SET WITH JARS' are connected.")
    else:
        print("User 701 and item 'JAM MAKING SET WITH JARS' are not connected.")
print("====================================")

if uidx2tidx_train.get(str(user_index)) is not None:
    if wrong_item_index_3 in uidx2tidx_train[str(user_index)]:
        print("User 701 and item 'WHITE HANGING HEART T-LIGHT HOLDER' are connected.")
    else:
        print("User 701 and item 'WHITE HANGING HEART T-LIGHT HOLDER' are not connected.")
print("====================================")

# 但是可以检查uidx2tidx，是否在701的item里有别的item，这个item连接到另一个user，然后另一个user的item里有这三个错误item
items_for_701 = uidx2tidx_train[str(user_index)]
num_of_connection_user_wrong = 0

for item in items_for_701:
    for user in uidx2tidx_train.keys():
        if user != str(user_index):
            if item in uidx2tidx_train[user]:
                if wrong_item_index_3 in uidx2tidx_train[user]: # and wrong_item_index_2 in uidx2tidx_train[user] and wrong_item_index_3 in uidx2tidx_train[user]:
                    # print("User 701 and item 'REGENCY CAKESTAND 3 TIER' and 'JAM MAKING SET WITH JARS' and 'WHITE HANGING HEART T-LIGHT HOLDER' are connected through another user.")
                    # print("And this user is: " + str(user))
                    num_of_connection_user_wrong += 1

print("User 701 and item 'REGENCY CAKESTAND 3 TIER' and 'JAM MAKING SET WITH JARS' and 'WHITE HANGING HEART T-LIGHT HOLDER' are connected through another user.")
print("And there are " + str(num_of_connection_user_wrong) + " users that connect them.")

User 701 and item 'REGENCY CAKESTAND 3 TIER' are not connected.
User 701 and item 'JAM MAKING SET WITH JARS' are not connected.
User 701 and item 'WHITE HANGING HEART T-LIGHT HOLDER' are not connected.
User 701 and item 'REGENCY CAKESTAND 3 TIER' and 'JAM MAKING SET WITH JARS' and 'WHITE HANGING HEART T-LIGHT HOLDER' are connected through another user.
And there are 1878 users that connect them.


In [255]:
# uidx为701的user的uid
for user in user_ids_kv.keys():
    if user_ids_kv[user] == 701:
        print(user)
        break

13308
